# NOTE:  THIS NOTEBOOK WILL TAKE 5-10 MINUTES TO COMPLETE.

# PLEASE BE PATIENT.


# 使用 SageMaker processing 和 Spark 分析数据质量

通常，机器学习 (ML) 过程由几个步骤组成。首先，使用各种 ETL 作业收集数据，然后对数据进行预处理，通过整合标准技术或先验知识来呈现数据集，最后使用算法训练机器学习模型。

通常，诸如 Spark 之类的分布式数据处理框架用于处理和分析数据集，以检测数据质量问题并为模型训练做好准备。  

在这本notebook中，我们将使用 Amazon SageMaker Processing 和一个名为 [**Deequ**](https://github.com/awslabs/deequ)的库，并利用 Spark 的强大功能和托管 SageMaker 处理任务来运行我们的数据处理工作负载。

以下是 Deequ 上的一些资源： 
* Blog Post:  https://aws.amazon.com/blogs/big-data/test-data-quality-at-scale-with-deequ/
* Research Paper:  https://assets.amazon.science/4a/75/57047bd343fabc46ec14b34cdb3b/towards-automated-data-quality-management-for-machine-learning.pdf

![Deequ](./img/deequ.png)

![Processing Job](./img/processing.jpg)

<a name='1'></a>
## Set up Kernel and Required Dependencies

First, check that the correct kernel is chosen.

<img src="img/kernel_set_up.png" width="300"/>

You can click on that to see and check the details of the image, kernel, and instance type.

<img src="img/w3_kernel_and_instance_type.png" width="600"/>

# Amazon Customer Reviews Dataset

https://s3.amazonaws.com/amazon-reviews-pds/readme.html

#### 数据列

- `marketplace`: 2 个字母的国家/地区代码（在本例中全部为 “US”)
- `customer_id`: 随机标识符，可用于汇总单个作者撰写的评论
- `review_id`: 评论的唯一 ID
- `product_id`: 亚马逊标准识别码 (ASIN)  `http://www.amazon.com/dp/<ASIN>` 指向商品详情页面的链接
- `product_parent`: 该 ASIN 的父商品。多个 ASIN（同一商品的颜色或格式变体）可以合并为一个父商品
- `product_title`: 商品的标题描述
- `product_category`: 可用于对评论进行分组的广泛产品类别（在本例中为数字视频）
- `star_rating`: 该评论的评分（1 到 5 星）
- `helpful_votes`: 评论的有用票数
- `total_votes`: 评论获得的总票数
- `vine`: 这篇评论是作为 [Vine](https://www.amazon.com/gp/vine/help) 计划的一部分写的吗？
- `verified_purchase`: 评论来自经过验证的购买吗？
- `review_headline`: 评论的标题
- `review_body`: 评论的文本
- `review_date`: 撰写评论的日期

In [2]:
%store -r setup_dependencies_passed

In [3]:
try:
    setup_dependencies_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOKS")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [4]:
print(setup_dependencies_passed)

True


# 使用带有 Spark 的 SageMaker 处理作业运行分析作业
使用亚马逊 SageMaker Python 软件开发工具包提交processiong任务。使用刚刚使用我们的 Spark 脚本构建的 Spark 容器。

In [5]:
# 导入SageMaker和Boto3模块
import sagemaker  
import boto3

# 创建一个SageMaker Session对象
sess = sagemaker.Session() 

# 获取默认的S3 bucket名称
bucket = sess.default_bucket()

# 获取执行角色
role = sagemaker.get_execution_role()  

# 获取当前AWS区域
region = boto3.Session().region_name

# 导入botocore模块
import botocore.config

# 创建botocore配置对象,添加user agent信息
config = botocore.config.Config(
    user_agent_extra='dsoaws/2.0'  
)

# 查看 Spark 预处理脚本

In [6]:
# pygmentize将源代码文件进行语法高亮,并输出到终端。
!pygmentize preprocess_deequ_pyspark.py

from __future__ import print_function
from __future__ import unicode_literals

import time
import sys
import os
import shutil
import csv
import subprocess

subprocess.check_call([sys.executable, "-m", "pip", "install", "--no-deps", "pydeequ==0.1.5"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas==1.1.4"])

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType
from pyspark.sql.functions import *

from pydeequ.analyzers import *
from pydeequ.checks import *
from pydeequ.verification import *
from pydeequ.suggestions import *

# PySpark Deequ GitHub Repo:  https://github.com/awslabs/python-deequ


def main():
    args_iter = iter(sys.argv[1:])
    args = dict(zip(args_iter, args_iter))

    # Retrieve the args and replace 's3://' with 's3a://' (used by Spark)
    s3_input_data = args["s3_input_data"].replace("s3://", "s3a://")
    print(s3_input_data)
    s3_output_analyze_dat

In [7]:
from sagemaker.spark.processing import PySparkProcessor

# 创建PySparkProcessor对象,用于在SageMaker上运行Spark作业
processor = PySparkProcessor(
    # 作业的基本名称
    base_job_name="spark-amazon-reviews-analyzer",
    # 作业执行角色
    role=role,
    # Spark版本
    framework_version="2.4",
    # 实例数量
    instance_count=2,
    # 实例类型
    instance_type="ml.m5.2xlarge",
    # 最大运行时间
    max_runtime_in_seconds=300,
)

In [10]:
s3_input_data = "s3://{}/amazon-reviews-pds/tsv/".format(bucket)
print(s3_input_data)

s3://sagemaker-us-east-1-941797585610/amazon-reviews-pds/tsv/


In [11]:
!aws s3 ls $s3_input_data

## Setup Output Data

In [ ]:
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

output_prefix = "amazon-reviews-spark-analyzer-{}".format(timestamp_prefix)
processing_job_name = "amazon-reviews-spark-analyzer-{}".format(timestamp_prefix)

print("Processing job name:  {}".format(processing_job_name))

In [ ]:
s3_output_analyze_data = "s3://{}/{}/output".format(bucket, output_prefix)

print(s3_output_analyze_data)

## Start the Spark Processing Job

_Notes on not using `ProcessingInput` and `ProcessingOutput`:_
* Since Spark natively reads/writes from/to S3 using s3a://, we can avoid the copy required by `ProcessingInput` and `ProcessingOutput` (FullyReplicated or ShardedByS3Key) and just specify the S3 input and output buckets/prefixes.
* See https://github.com/awslabs/amazon-sagemaker-examples/issues/994 for issues related to using /opt/ml/processing/input/ and output/
* If we use `ProcessingInput`, the data will be copied to each node (which we don't want in this case since Spark already handles this)

In [ ]:
from sagemaker.processing import ProcessingOutput

processor.run(
    submit_app="preprocess_deequ_pyspark.py",
    submit_jars=["deequ-1.0.3-rc2.jar"],
    arguments=[
        "s3_input_data",
        s3_input_data,
        "s3_output_analyze_data",
        s3_output_analyze_data,
    ],
    logs=True,
    wait=False,
)

In [ ]:
from IPython.core.display import display, HTML

processing_job_name = processor.jobs[-1].describe()["ProcessingJobName"]

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, processing_job_name
        )
    )
)

In [ ]:
from IPython.core.display import display, HTML

processing_job_name = processor.jobs[-1].describe()["ProcessingJobName"]

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After a Few Minutes</b>'.format(
            region, processing_job_name
        )
    )
)

In [ ]:
from IPython.core.display import display, HTML

s3_job_output_prefix = output_prefix

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Spark Job Has Completed</b>'.format(
            bucket, s3_job_output_prefix, region
        )
    )
)

# Monitor the Processing Job

In [ ]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(
    processing_job_name=processing_job_name, sagemaker_session=sess
)

processing_job_description = running_processor.describe()

print(processing_job_description)

In [ ]:
running_processor.wait()

# _Please Wait Until the ^^ Processing Job ^^ Completes Above._

# Inspect the Processed Output 

## These are the quality checks on our dataset.

## _The next cells will not work properly until the job completes above._

In [ ]:
!aws s3 ls --recursive $s3_output_analyze_data/

## Copy the Output from S3 to Local
* dataset-metrics/
* constraint-checks/
* success-metrics/
* constraint-suggestions/


In [ ]:
!aws s3 cp --recursive $s3_output_analyze_data ./amazon-reviews-spark-analyzer/ --exclude="*" --include="*.csv"

## Analyze Constraint Checks

In [ ]:
import glob
import pandas as pd

def load_dataset(path, sep, header):
    data = pd.concat(
        [pd.read_csv(f, sep=sep, header=header) for f in glob.glob("{}/*.csv".format(path))], ignore_index=True
    )

    return data

In [ ]:
df_constraint_checks = load_dataset(path="./amazon-reviews-spark-analyzer/constraint-checks/", sep="\t", header=0)
df_constraint_checks[["check", "constraint", "constraint_status", "constraint_message"]]

## Analyze Dataset Metrics

In [ ]:
df_dataset_metrics = load_dataset(path="./amazon-reviews-spark-analyzer/dataset-metrics/", sep="\t", header=0)
df_dataset_metrics

## Analyze Success Metrics

In [ ]:
df_success_metrics = load_dataset(path="./amazon-reviews-spark-analyzer/success-metrics/", sep="\t", header=0)
df_success_metrics

## Analyze Constraint Suggestions

In [ ]:
pd.set_option('max_colwidth', 999)

df_constraint_suggestions = load_dataset(path='./amazon-reviews-spark-analyzer/constraint-suggestions/', sep='\t', header=0)
df_constraint_suggestions

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>